## parameter 최적화

모델 학습과정
== 각 반복단계에서 모델은 출력을 추즉하고, 추측과 정답 사이의 loss를 계산하고, loss의 derivative를 수집하고, 경사하강법을 사용하여 parameter를 optimze함

In [13]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Hyperparameter

* epoch
* batch size
* learning rate

In [14]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop

최적화 단계의 각 반복 iteration을 에포크라고 부른다.

하나의 에폭은 다음 두부분으로 구성된다.

* train loop : 학습용 데이터셋을 반복하고 최적ㅇ의 매개변수로 수렴
* validation/test loop : 모델의 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 interate(반복)

In [15]:
# Loss Function
# 손실함수 초기화

loss_fn = nn.CrossEntropyLoss()

In [16]:
# Optimizer
# 모든 최적화 절차는 optimizer 객체에 encapsulate 됨.
# parameter와 learning rate, 하이퍼파라미터를 등록하여 옵티마이저를 초기화

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

학습단계에서 최적화
* optimizer.zero_grad() : parameter의 gradient 재설정, 명시적으로 0으로 설정
* loss.backward()를 하여 prediction loss를 역전파
* optimizer.step()을 호출하여 역전파 단계에서 수집된 gradient를 사용하여 parameter 조정

In [17]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # 모델을 학습(train) 모드로 설정합니다 - 배치 정규화(Batch Normalization) 및 드롭아웃(Dropout) 레이어들에 중요합니다.
    # 이 예시에서는 없어도 되지만, 모범 사례를 위해 추가해두었습니다.
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # 모델을 평가(eval) 모드로 설정합니다 - 배치 정규화(Batch Normalization) 및 드롭아웃(Dropout) 레이어들에 중요합니다.
    # 이 예시에서는 없어도 되지만, 모범 사례를 위해 추가해두었습니다.
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # torch.no_grad()를 사용하여 테스트 시 변화도(gradient)를 계산하지 않도록 합니다.
    # 이는 requires_grad=True로 설정된 텐서들의 불필요한 변화도 연산 및 메모리 사용량 또한 줄여줍니다.
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [18]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.312815  [   64/60000]
loss: 2.298903  [ 6464/60000]
loss: 2.280417  [12864/60000]
loss: 2.273459  [19264/60000]
loss: 2.255248  [25664/60000]
loss: 2.220623  [32064/60000]
loss: 2.235696  [38464/60000]
loss: 2.198894  [44864/60000]
loss: 2.197630  [51264/60000]
loss: 2.168950  [57664/60000]
Test Error: 
 Accuracy: 32.6%, Avg loss: 2.165024 

Epoch 2
-------------------------------
loss: 2.176075  [   64/60000]
loss: 2.166766  [ 6464/60000]
loss: 2.111370  [12864/60000]
loss: 2.128442  [19264/60000]
loss: 2.084311  [25664/60000]
loss: 2.011787  [32064/60000]
loss: 2.050544  [38464/60000]
loss: 1.967912  [44864/60000]
loss: 1.973341  [51264/60000]
loss: 1.905847  [57664/60000]
Test Error: 
 Accuracy: 57.8%, Avg loss: 1.905341 

Epoch 3
-------------------------------
loss: 1.933892  [   64/60000]
loss: 1.903492  [ 6464/60000]
loss: 1.790928  [12864/60000]
loss: 1.833436  [19264/60000]
loss: 1.731050  [25664/60000]
loss: 1.668876  [32064/600